<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex10notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex10notebookC

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

----
## マハラノビス距離 / 正規分布の当てはめの応用
---

ex10notebookA 「正規分布のパラメータの推定」の項では，最尤推定という手法によって，データに正規分布を当てはめられる（正規分布の平均と分散共分散行列を推定できる）ことを学びました．データに確率分布を当てはめるこのような方法は，様々な場面で応用がききます．この notebook の後半でそのような応用事例を紹介するのですが，その話の導入として，「マハラノビス距離」というものを説明します．ここで説明していることは，次回以降の内容である「判別分析」でも活用することになります．

<b><font color="#ff0000">
注意:
今回の notebook の中には，コードセルを実行すると問題の解答が表示されるようになっている箇所があります．
</font>
</b>


In [ ]:
# 必要なパッケージのインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy のもろもろ
import scipy as sp
from scipy.spatial import distance
from scipy.stats import norm, multivariate_normal

# Python による「コンピュータビジョン(Computer Vision)」のためのライブラリ OpenCV のパッケージをインポート
import cv2

# 「解答」を示す際に文字列を復号するのに使う
import base64
# 復号した文字列を Markdown 形式で（数式は LaTeX でフォーマットして）表示
from IPython.display import display, Markdown

---
### 正規分布に対する尤度とマハラノビス距離

マハラノビス距離とはどんなものか，以前にも使った，この授業の第7回までの Quiz の得点率 [%]（Quiz）と先日の小テストの得点率 [%]（Exam）のデータを使って説明します．

In [ ]:
##### CSV ファイルを読み込む #####
URL = 'https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA2024-QvsE1107.csv'
df = pd.read_csv(URL, index_col=0)
X = df.to_numpy()
N, D = X.shape
print(f'N = {N}, D = {D}')
#df

このデータに2次元正規分布を当てはめる（平均と分散共分散行列を推定する）と，次のような結果となります．

In [ ]:
# 平均 mu 分散共分散行列 cov を推定
mu = np.mean(X, axis=0)
print(f'mu = {mu}')
Xd = X - mu
cov = Xd.T @ Xd / N
print('cov = ')
print(cov)

# 2点 P, Q
P = np.array([90, 90])
Q = np.array([40, 90])

# 確率密度描画のためのグリッドデータの作成
xmin, xmax = -10, 110
xx, yy = np.mgrid[xmin:xmax:1.0, xmin:xmax:1.0]
zz = multivariate_normal.pdf(np.dstack((xx, yy)), mean=mu, cov=cov)

# データの散布図と正規分布の確率密度関数を重ねて描く
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X[:, 0], X[:, 1], s=10)
ax.scatter(P[0], P[1], s=100, marker='*', label=f'P({P[0]},{P[1]})')
ax.scatter(Q[0], Q[1], s=100, marker='*', label=f'Q({Q[0]},{Q[1]})')
tmp = np.vstack((mu, P))
ax.plot(tmp[:, 0], tmp[:, 1], linestyle='-', color='gray')
tmp = np.vstack((mu, Q))
ax.plot(tmp[:, 0], tmp[:, 1], linestyle='-', color='gray')
ax.contour(xx, yy, zz, colors=['#ffa0a0', '#ff5050', '#ff0000'], levels=[0.00002, 0.0001, 0.0005])
ax.plot(mu[0], mu[1], '+', markersize=16, color='r')
ax.axvline(mu[0], linestyle='dotted', color='gray')
ax.axhline(mu[1], linestyle='dotted', color='gray')
ax.axvline(0, color='gray')
ax.axhline(0, color='gray')
ax.set_xlim(xmin, xmax)
ax.set_ylim(xmin, xmax)
ax.set_xlabel('Quiz')
ax.set_ylabel('Exam')
ax.set_aspect('equal')
ax.legend()
plt.show()

この散布図には，正規分布の確率密度関数を可視化するための同心楕円と，2つの点 $P(90, 90), Q(40, 90)$ も描かれています．
2つの線分は，これらの点と正規分布の平均との間を結んでいます．

この2点 $P, Q$ は，平均から同じくらい離れているように見えます．$P, Q$ がそれぞれ平均とどのくらい離れているかを測るため，ユークリッド距離（最も一般的な距離）を計算してみましょう．ある点 $\pmb{x}$ と $\pmb{y}$ の間のユークリッド距離は $\|\pmb{x} - \pmb{y}\|$ と表されます．

In [ ]:
# 平均と点PおよびQの間のユークリッド距離
v = mu - P
edistP = np.sqrt(v @ v)
v = mu - Q
edistQ = np.sqrt(v @ v)
print(f'平均と点 P({P[0]},{P[1]}) の間のユークリッド距離 = {edistP:.2f}')
print(f'平均と点 Q({Q[0]},{Q[1]}) の間のユークリッド距離 = {edistQ:.2f}')

ユークリッド距離を測ると，$P, Q$ は平均からほぼ同じくらい離れていることが分かります．
しかし，図を見ると，点 $P$ は散らばったデータ点たちに近い所に位置している一方，点 $Q$ はそれらから少し離れています．
したがって，このデータのサンプルとしては，点 $Q$ よりも点 $P$ の方がより適当なように思えます．最尤推定の項で説明したように，このことは，**尤度** によって定量的に測ることができます．

平均 $\pmb{\mu}$，分散共分散行列 $\Sigma$ の $D$ 次元正規分布の確率密度関数を $f(\pmb{x})$ とおくと，

$$
f(\pmb{x}) =
 \frac{1}{\sqrt{(2\pi)^D|\Sigma|}}\exp\left(-\frac{1}{2}(\pmb{x}-\pmb{\mu})^{\top}\Sigma^{-1}(\pmb{x}-\pmb{\mu})\right) \qquad (1)
$$

です．この正規分布に対する $\pmb{x}$ の尤度を $\ell(\pmb{x})$ とおくと，$\ell(\pmb{x}) = f(\pmb{x})$ ですが，対数をとった値の方が扱いやすいので，**対数尤度** を求めると，

$$
\log\ell(\pmb{x}) = -\frac{D}{2}\log{(2\pi)} -\frac{1}{2}\log{|\Sigma|} - \frac{1}{2}(\pmb{x}-\pmb{\mu})^{\top}\Sigma^{-1}(\pmb{x}-\pmb{\mu}) \qquad (2)
$$

となります．実際に，2点 $P,Q$ の対数尤度を計算すると，次のようになります．

In [ ]:
# 推定された正規分布に対する点P, Qの対数尤度
llP = multivariate_normal.logpdf(P, mean=mu, cov=cov)
llQ = multivariate_normal.logpdf(Q, mean=mu, cov=cov)
print(f'推定された正規分布に対する点 P({P[0]},{P[1]}) の対数尤度 = {llP:.2f}')
print(f'推定された正規分布に対する点 Q({Q[0]},{Q[1]}) の対数尤度 = {llQ:.2f}')

平均との間のユークリッド距離では点 $P, Q$ にほとんど差がありませんでしたが，対数尤度を測ると，$Q$ よりも $P$ の方がその値が大きく，この正規分布から生成された標本としてより尤もらしいことが分かります．このように，尤度/対数尤度を用いると「対象とする正規分布から生成されたことの尤もらしさ」を評価することができます．

さて，ここで式 $(2)$ をよく見ると，$\mu, \Sigma$ が固定されているとき、この対数尤度は次のような形をしています．

$$
\log\ell(\pmb{x}) = - \frac{1}{2}(\pmb{x}-\pmb{\mu})^{\top}\Sigma^{-1}(\pmb{x}-\pmb{\mu}) + (定数) \qquad (3)
$$

$(定数)$ の項は $\pmb{x}$ によらず一定です．したがって，ある一つの正規分布に対する異なる2点の尤度の大小を比較するという目的においては，

$$
(\pmb{x}-\pmb{\mu})^{\top}\Sigma^{-1}(\pmb{x}-\pmb{\mu}) \qquad (4)
$$

という量を計算すればよいことが分かります．ただし，式$(3)$にあった負号がなくなっているので，式$(4)$の値が小さい方が尤度が大きくなります．

以上をふまえて，ユークリッド距離に代わる新しい距離の規準として，次式で定義される **マハラノビス距離** (Mahalanobis' distance) （注）を導入します．

$$
d_M(\pmb{x}) = \sqrt{(\pmb{x}-\pmb{\mu})^{\top}\Sigma^{-1}(\pmb{x}-\pmb{\mu})} \qquad (5)
$$

このマハラノビス距離は，点 $\pmb{x}$ が，平均 $\pmb{\mu}$ 分散共分散行列 $\Sigma$ の正規分布からどのくらい離れているかを表す量です．特に，分散共分散行列 $\Sigma$ が単位行列に等しい場合，$d_M(\pmb{x}) = \sqrt{(\pmb{x}-\pmb{\mu})^{\top}(\pmb{x}-\pmb{\mu})} = \sqrt{\|\pmb{x} - \pmb{\mu}\|^2} = \|\pmb{x} - \pmb{\mu}\|$ となり，$\pmb{x}$ と $\pmb{\mu}$ の間のユークリッド距離に一致します．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※ 注意: 余談ですが，マハラノビスは統計学者の名前です．
Prasanta Chandra Mahalanobis（プラサンタ・チャンドラ・マハラノビス）：  インドの統計学者， 1893 - 1972，<a href="https://ja.wikipedia.org/wiki/%E3%83%97%E3%83%A9%E3%82%B5%E3%83%B3%E3%82%BF%E3%83%BB%E3%83%81%E3%83%A3%E3%83%B3%E3%83%89%E3%83%A9%E3%83%BB%E3%83%9E%E3%83%8F%E3%83%A9%E3%83%8E%E3%83%93%E3%82%B9">Wikipedia</a>
</span>

上で推定された正規分布に対する点 $P, Q$ のマハラノビス距離の値を求めてみると，次のようになります．

In [ ]:
# 推定された正規分布に対する点P, Qのマハラノビス距離
covInv = np.linalg.inv(cov)
mdistP = distance.mahalanobis(P, mu, covInv)
mdistQ = distance.mahalanobis(Q, mu, covInv)
print(f'推定された正規分布に対する点 P({P[0]},{P[1]}) のマハラノビス距離 = {mdistP:.2f}')
print(f'推定された正規分布に対する点 Q({Q[0]},{Q[1]}) のマハラノビス距離 = {mdistQ:.2f}')

対数尤度の大小を反映して，点$P$の方がマハラノビス距離が小さくなっています．



#### 問題1

3つの2次元ベクトル（$2\times 1$ 行列） $\pmb{x}, \pmb{y}, \pmb{\mu}$ を

$$
\pmb{x} = \begin{pmatrix} 4 \\ 3 \end{pmatrix}\qquad
\pmb{y} = \begin{pmatrix} 5 \\ 5 \end{pmatrix}\qquad
\pmb{\mu} = \begin{pmatrix} 3 \\ 4 \end{pmatrix}
$$

とおく．また，行列 $\Sigma$ を

$$
\Sigma = \begin{pmatrix} 2 & 1 \\ 1 & 1 \end{pmatrix}
$$

とおく．このとき，次の問に答えなさい．

(1) $\pmb{x}$ と $\pmb{\mu}$ との間のユークリッド距離 $\| \pmb{x} - \pmb{\mu}\|$ および $\pmb{y}$ と $\pmb{\mu}$ との間のユークリッド距離 $\|\pmb{y}-\pmb{\mu}\|$ を求め，この距離規準では $\pmb{x}$ と $\pmb{y}$ のどちらの方が $\pmb{\mu}$ に近いか答えなさい．

(2) 式$(5)$の定義を用いて，平均 $\pmb{\mu}$，分散共分散行列 $\Sigma$ の正規分布に対する $\pmb{x}, \pmb{y}$ のマハラノビス距離 $d_M(\pmb{x}), d_M(\pmb{y})$ を求め，この距離規準では $\pmb{x}$ と $\pmb{y}$ のどちらの方がこの正規分布に近いか（この正規分布から生成されたデータとして尤もらしいか）答えなさい．



次のコードを実行すると，上記の略解を表示します．また，上記問題の状況を視覚的に表示させることができます．

In [ ]:
# 正規分布の平均と分散共分散行列
mu = np.array([3.0, 4.0])
cov = np.array([[2.0, 1.0], [1.0, 1.0]])
# 2点 R, S
R = np.array([4, 3])
S = np.array([5, 5])

fig, ax = plt.subplots(figsize=(6, 6))
xmin, xmax = -1, 7
ymin, ymax = -1, 7
xx, yy = np.mgrid[xmin:xmax:0.1, xmin:xmax:0.1]
# 等高線を描くマハラノビス距離の値
levels = [0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
X = np.dstack((xx, yy)).reshape((-1, 2))
d2 = np.sum(((X - mu) @ np.linalg.inv(cov)) * (X - mu), axis=1)
zz = np.sqrt(d2).reshape((xx.shape[0], yy.shape[0]))
cs = ax.contour(xx, yy, zz, cmap='Blues_r', levels=levels)
ax.clabel(cs)
ax.scatter(R[0], R[1], s=100, marker='*', label=r'$\mathbf{x}$')
ax.scatter(S[0], S[1], s=100, marker='*', label=r'$\mathbf{y}$')
ax.scatter(mu[0], mu[1], s=100, marker='+', color='blue', label=r'$\mathbf{\mu}$')
ax.axhline(0, color='gray')
ax.axvline(0, color='gray')
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
ax.legend()
plt.show()

# ユークリッド距離とマハラノビス距離
tmp = R - mu
edistR = np.sqrt(tmp @ tmp)
tmp = S - mu
edistS = np.sqrt(tmp @ tmp)
covInv = np.linalg.inv(cov)
mdistR = distance.mahalanobis(R, mu, covInv)
mdistS = distance.mahalanobis(S, mu, covInv)
print(f'点 x と平均のユークリッド距離 = {edistR:.4f}   マハラノビス距離 = {mdistR:.4f}')
print(f'点 y と平均のユークリッド距離 = {edistS:.4f}   マハラノビス距離 = {mdistS:.4f}')

Q = b'CigxKSAkXHwgXHBtYnt4fSAtIFxwbWJ7XG11fVx8ID0gXHNxcnR7KDQtMyleMisoMy00KV4yfSA9IFxzcXJ0ezJ9LFwgIFx8IFxwbWJ7eX0gLSBccG1ie1xtdX1cfCA9IFxzcXJ0ezV9JO+8jiRccG1ie3h9JCDjga7mlrnjgYzov5HjgYTvvI4KCigyKSAkXFNpZ21hXnstMX0gPSBcYmVnaW57cG1hdHJpeH0gMSAmIC0xIFxcIC0xICYyIFxlbmR7cG1hdHJpeH0kIOOCiOOCiu+8jAokZF9NXjIoXHBtYnt4fSkgPSBcYmVnaW57cG1hdHJpeH0gNCAtIDMgJiAzIC0gNFxlbmR7cG1hdHJpeH1cYmVnaW57cG1hdHJpeH0gMSAmIC0xIFxcIC0xICYyIFxlbmR7cG1hdHJpeH1cYmVnaW57cG1hdHJpeH0gNC0zIFxcIDMtNCBcZW5ke3BtYXRyaXh9ID0gMiTvvIzjgojjgaPjgaYgJGRfTShccG1ie3h9KSA9IFxzcXJ0ezV9JO+8jgrlkIzmp5jjgavvvIwkZF9NKFxwbWJ7eX0pID0gXHNxcnR7Mn0k77yOJFxwbWJ7eX0kIOOBruaWueOBjOi/keOBhO+8jgo='
display(Markdown(base64.b64decode(Q).decode('utf-8')))

---
### 正規分布の当てはめの応用

データに正規分布を当てはめて，この正規分布に対する個々のデータ点の尤度やマハラノビス距離を求めると，それらがこの正規分布から生成されたことの尤もらしさを評価できます．あるデータのマハラノビス距離がとても大きい（尤度がとても小さい）場合，そのデータは，通常のデータの分布から外れた「異常な」ものと判断できそうです．このような考えに基づいて，次のような手順で「異常検出」/「異常検知」（anomaly detection）の仕組みを作ることができます．

1. データを集める．例えば，多数の人々の医療検査のデータ，稼働中の工業機械のデータ（様々な場所の温度，圧力，回転数，etc.）など．
1. 集めたデータに正規分布を当てはめる．
1. 正常/異常を判断するためのマハラノビス距離のしきい値を定める．
1. 新しいデータのマハラノビス距離を測り，その値がしきい値より小さければ正常，大きければ異常と判断する

#### 画像の顔らしさの数値化

「異常検出」とは少し異なりますが，顔の画像をたくさん集めて正規分布を当てはめ，マハラノビス距離によって画像の「顔らしさ」を測る実験をやってみましょう．次のような手順です．

1. 顔画像の位置や大きさを揃えて $64\times 64$ 画素にしたものを大量に用意する．ひとつの顔画像のデータを $\pmb{x}_n$ （$4096\times 1$ 行列，$n = 1, 2, \ldots, N$）と表す（グレイスケール画像なのでデータの次元数は $64\times 64 = 4096$）．
1. 次元削減のため $\{ \pmb{x}_n \}$ に主成分分析を適用．累積寄与率 0.99 以上となる最小の次元数は $700$ だったため，$\{ \pmb{x}_n \}$ の分散共分散行列の大きい方の固有値 $\lambda_1, \lambda_2, \ldots \lambda_{700}$ に対応する固有ベクトルをこの順にならべた $4096\times 700$ の行列 $U$ をつくる．
1. 次元削減したデータ $\pmb{y}_n = U^{\top}(\pmb{x}_n - \pmb{\mu})$ （$\pmb{\mu}$ は $\{ \pmb{x}_n \}$ の平均）の平均は $\pmb{0}$ で分散共分散行列は $\textrm{diag}(\lambda_1, \lambda_2, \ldots, \lambda_{700})$ なので，これらのデータが正規分布に従うと仮定すると，この正規分布に対するマハラノビス距離は
$$
d_M(\pmb{y}) = \pmb{y}^{\top}
\begin{pmatrix}
\lambda_1 & & 0\\
& \ddots & \\
0 & & \lambda_{700}
\end{pmatrix}\pmb{y} = \frac{y_1^2}{\lambda_1} + \frac{y_2^2}{\lambda_2} + \cdots + \frac{y_{700}^2}{\lambda_{700}}
$$
となる．
1. 画像 $\pmb{x}$ が与えられたら，$\pmb{y} = U^{\top}(\pmb{x} - \pmb{\mu})$ として $d_M(\pmb{y})$ をその画像の「顔らしさ」とする．

上記のうち 1. と 2. の手順はあらかじめ行ってあります．以下では，そこで得られた $\pmb{\mu}, U, \lambda_1, \ldots, \lambda_{700}$ をファイルから読み込んで，4. の計算を行うようになっています．

この実験では，[LFW (Labeled Faces in the Wild)](https://vis-www.cs.umass.edu/lfw/) という顔画像データセットを用いています．およそ1.3万枚の顔画像から成っています．

In [ ]:
# takataka のウェブサイトからデータを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/faciallikelihood.npz
import os
path = 'faciallikelihood.npz'
if os.path.exists(path):
    FL = np.load(path)
else:
    print(f'ファイル {path} の読み込みに失敗したようです．再実行してみてください')

In [ ]:
# PCA のパラメータ（平均，固有ベクトル，固有値）
mu, U, lam = FL['mu'], FL['U'], FL['lam']
print(f'mu.shape= {mu.shape}')
print(f'U.shape = {U.shape}')
print(f'lam.shape = {lam.shape}')

次のセルを実行すると，様々なサンプル画像に対して顔らしさを計算し，それらを表示します．

In [ ]:
# サンプル画像
samples = FL['samples']
X = samples.reshape((samples.shape[0], -1)) / 255
# 主成分分析による次元削減
Y = (X - mu) @ U.T
# 推定された正規分布に対するマハラノビス距離の計算
dM = np.sqrt(np.sum((Y**2 / lam), axis=1))
# 画像とそのマハラノビス距離の表示
fig, ax = plt.subplots(4, 6, figsize=(6, 5))
for i in range(4):
    for j in range(6):
        k = i*6 + j
        ax[i, j].imshow(samples[k], cmap='gray', vmin=0, vmax=255)
        ax[i, j].axis('off')
        ax[i, j].set_title(f'{dM[k]:.1f}', fontsize=16)
fig.tight_layout()
plt.show()

左上の画像は，パラメータの推定に用いたデータの平均 $\pmb{\mu}$ です．定義から明らかなように，この画像のマハラノビス距離は $0$ です．

左上の画像を除いた上から3行の17枚の画像は，パラメータの推定に用いたデータの一部を，マハラノビス距離の小さい方から順にならべたものです．
画像を見ると分かるように，「顔らしさ」とは言いつつも，顔の向きが正面でない，濃い陰がある，メガネやゴーグルをかけている等の，顔そのもののつくりとは異なる要素のために顔らしさの数値が小さくなっているようです．

最下行は，ヒトの顔ではない画像を用いた実験の結果です．左から，MVAのロゴ，ネコの顔画像3枚，顔のイラスト（「いらすとや」の素材を利用させていただきました），アルチンボルド作の絵画（注）です．

見ての通り，あまり良い結果は得られませんでした．画素値をそのまま用いる方法であるため，顔そのもののつくりを表す特徴と，顔の向きや陰影，アクセサリ等の特徴を区別できていないことが原因と考えられます．機械学習の技術を用いて顔本来の特徴をうまく抽出できるようにすると，改善できるかもしれません．その辺りのことは，学部3年の「機械学習I/II」で少しだけ，大学院の「機械学習特論I/II」などでしっかり学べるかも．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※ 注意: <a href="https://ja.wikipedia.org/wiki/%E3%82%B8%E3%83%A5%E3%82%BC%E3%83%83%E3%83%9A%E3%83%BB%E3%82%A2%E3%83%AB%E3%83%81%E3%83%B3%E3%83%9C%E3%83%AB%E3%83%89">アルチンボルド</a> の「<a href="https://ja.wikipedia.org/wiki/%E3%82%A6%E3%82%A7%E3%83%AB%E3%83%88%E3%82%A5%E3%83%A0%E3%83%8C%E3%82%B9%E3%81%A8%E3%81%97%E3%81%A6%E3%81%AE%E7%9A%87%E5%B8%9D%E3%83%AB%E3%83%89%E3%83%AB%E3%83%952%E4%B8%96%E5%83%8F">ウェルトゥムヌスとしての皇帝ルドルフ2世像</a>」．アルチンボルドは，野菜・果物や花などを組み合わせた肖像画でよく知られた16世紀の画家．
</span>

以下では，自分で用意した顔画像で顔らしさを計算できます．

In [ ]:
# 顔検出器の準備
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')

# Colab へのファイルアップロードを実行する関数
#
def uploadToColab():
    try:
        from google.colab import files
        rv = files.upload()
    except:
        print('このコードは Colab 以外の環境では実行できないよ．')

# OpenCVの形式の画像を表示する関数
#
def imshow(img):
    try:
        from google.colab.patches import cv2_imshow
        cv2_imshow(img) # Colab上で実行している場合
    except:
        cv2.imshow(img)  # それ以外の場合

# 与えられた画像の中から顔を検出する
#
def faceDetector(img, faceCascade, maxSize=400):

    h, w = img.shape[:2]

    # (短辺の長さ) <= maxSize にリサイズ
    if min(w, h) > maxSize:
        if w <= h:
            w2, h2 = maxSize, h*maxSize//w
        else:
            w2, h2 = w*maxSize//h, maxSize
        imgDisp = cv2.resize(img, (w2, h2))
    else:
        imgDisp = np.copy(img)

    # 顔検出を実行
    imgGray = cv2.cvtColor(imgDisp, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(imgGray, 1.05, 4)
    nFace = len(faces)

    # 検出した顔のうち最初のひとりを選んで切り取り
    if nFace > 0:
        print(faces)
        x, y, ww, hh = faces[0]
        imgFace = np.copy(imgDisp[y:y+hh, x:x+ww, :])
        cv2.rectangle(imgDisp, (x, y), (x+ww, y+hh), color=(0, 255, 0), thickness=2)
        return nFace, imgDisp, imgFace
    else:
        return nFace, imgDisp, None


(1) 画像を Colab へアップロードします．次のような画像にしてください

- ひとりのひとのほぼ正面を向いた顔全体が写ってる．複数人を検出した場合，ひとりだけ抜き出します（あとで条件に合わない画像をわざと与えて実験してみるのもよいでしょう）．
- 画像全体に顔がドアップで写っているような場合はうまく顔検出ができないかもしれません．顔がもう少し小さく写ってる画像を探してみましょう．
- ファイル名に空白やマルチバイト文字（日本語など）が含まれているとうまく動作しない場合があるので，自分のPCの方で適当な名前に変えておく（拡張子は変えてはいけない）．

In [ ]:
# Colab へファイルをアップロード
uploadToColab()

# ls コマンドでファイルを一覧
! ls

上のセルを実行してファイルをアップロードできたら，次のセルのファイル名の部分にその名前を指定して，読み込ませましょう．

In [ ]:
# 画像を読み込む．`hoge.jpg` を自分がアップロードしたファイルの名前に修正
img = cv2.imread('hoge.png')
if img is None:
    print('画像を読み込めませんでした．やり直してください')

(2) 顔検出を実行して顔領域を切り取った画像を作る

In [ ]:
# 顔を検出する
numFace, imgDisp, imgFace = faceDetector(img, faceCascade, maxSize=400)
if numFace > 0:
    imshow(imgDisp)
    imshow(imgFace)
else:
    imshow(imgDisp)
    print('顔を検出できませんでした')

(3) 顔を所定の大きさにリサイズしてから次元削減し，マハラノビス距離を計算．

In [ ]:
# 使用する画像の選択
imgInput = imgFace  # 顔検出して切り取ったものを使用
#imgInput = img     # 読み込んだ画像をそのまま使用

# 顔画像を 64 x 64 にリサイズ
imgInput2 = cv2.resize(cv2.cvtColor(imgInput, cv2.COLOR_BGR2GRAY), (64, 64))
# 4096 次元ベクトルにする
xvec = imgInput2.reshape(-1) / 255
# PCAを利用した次元削減
yvec = (xvec - mu) @ U.T
# マハラノビス距離の計算
dM = np.sqrt(np.sum((yvec**2 / lam)))
# 画像を表示
imshow(imgInput2)
# 結果の表示
print(f'顔らしさ {dM:.2f}')

上記の実験では，顔検出を経由するため，顔検出ができないような画像で試すことができません．顔でない画像で試したいひとは，次のように実行してみてね．

1. (1) を普通に実行
1. (2) は実行しない
1. (3) のコードセル上部を次のように修正（`#`の場所を変える）して実行する
```
#imgInput = imgFace  # 顔検出して切り取ったものを使用
imgInput = img     # 読み込んだ画像をそのまま使用
```

ただし，入力した画像全体を 64 x 64 に縮小して用いることになります．画像の一部を使い対場合は，アップロードする前に手元で画像を加工するか，画像の一部を切り取るコードを追加する必要があります．後者を試したいひとは takataka に相談してね．